In [ ]:
!pip install transformers
!pip install nmslib
import torch
from transformers import AutoTokenizer, AutoModel
import json
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel
from scipy import spatial
import heapq
from numpy import dot
from numpy.linalg import norm
from scipy.spatial import distance
import numpy as np
from scipy.spatial import distance
from sklearn.metrics import ndcg_score
import nmslib
import glob
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import operator

# device = 'cpu' ''' keeping it all on cpu'''#torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
model = RobertaModel.from_pretrained("microsoft/codebert-base")

# Loading the tensors with embeddings for Code and Docstring

The 'training_data_length' variable below decides the size of training data for our experiments. This variable can be changed to conduct experiment with different sizes of training data

In [ ]:
training_data_length = 15000
test_data_length= 2000

train_doc_embds = torch.load('../CodeBERT_embeddings/train_embeddings/doc_embds_20000.pt')
train_code_embds = torch.load('../CodeBERT_embeddings/train_embeddings/code_embds_20000.pt')

train_code_embds = train_code_embds.detach().numpy() if torch.is_tensor(train_code_embds) else train_code_embds
train_code_embds = train_code_embds[:training_data_length]

train_doc_embds = train_doc_embds.detach().numpy() if torch.is_tensor(train_doc_embds) else train_doc_embds
train_doc_embds = train_doc_embds[:training_data_length]

train_code = train_code[:training_data_length]
train_doc = train_doc[:training_data_length]



test_doc_embds = torch.load('../CodeBERT_embeddings/test_embeddings/doc_embds_2000.pt')
test_code_embds = torch.load('../CodeBERT_embeddings/test_embeddings/code_embds_2000.pt')




test_doc_embds = test_doc_embds.detach().numpy()if torch.is_tensor(test_doc_embds) else test_doc_embds
test_code_embds = test_code_embds.detach().numpy()if torch.is_tensor(test_code_embds) else test_code_embds


test_doc_embds =test_doc_embds[:test_data_length]
test_code_embds = test_code_embds[:test_data_length]


print('train code examples size --',train_code_embds.shape)
print('train doc examples size --',train_doc_embds.shape)
print('test code examples size --',test_code_embds.shape)
print('test doc examples size --',test_doc_embds.shape)


train code examples size -- (5000, 768)
train doc examples size -- (15000, 768)
test code examples size -- (2000, 768)
test doc examples size -- (2000, 768)


In [ ]:
# finding indexes of docstring with different lengths for experiments 

less10 = []
from10to15 = []
great15 = []
for indx, i in enumerate(test_doc):
  if len(i.split())<11:
    less10.append(indx)
  elif 10<len(i.split())<16:
    from10to15.append(indx)
  elif len(i.split())>100:
    great15.append(indx)

less10 = less10[:400]
great15 = great15[:400]


print(len(less10))
print(len(from10to15))
print(len(great15))

In [ ]:
# finding train docstring which is most similar  to test docstring 
# this snippet creates a dictinory which acts as the true label for MRR
# we compare test docstring's embeddings with embeddings of all the docstrings which are preesent in training data

closest_doc_indexes = {}

for index, test_doc_embd in enumerate(test_doc_embds):
  test_doc_embd = np.reshape(test_doc_embd,(1,-1))
  similarities = cosine_similarity(test_doc_embd,train_doc_embds)[0]
  
  most_similar_train_doc = np.argmax(similarities)
  closest_doc_indexes[index] = most_similar_train_doc

print(closest_doc_indexes)

{0: 11192, 1: 2021, 2: 8007, 3: 13080, 4: 11006, 5: 13316, 6: 3732, 7: 3732, 8: 5985, 9: 4974, 10: 11005, 11: 13080, 12: 8564, 13: 5799, 14: 7623, 15: 6295, 16: 5828, 17: 8477, 18: 12535, 19: 5192, 20: 11744, 21: 5799, 22: 7520, 23: 13490, 24: 9646, 25: 5044, 26: 4757, 27: 11192, 28: 1474, 29: 5212, 30: 9065, 31: 9411, 32: 11005, 33: 1474, 34: 4635, 35: 8218, 36: 4690, 37: 5189, 38: 11283, 39: 8443, 40: 12609, 41: 4061, 42: 8443, 43: 8443, 44: 4617, 45: 8049, 46: 7520, 47: 5799, 48: 13066, 49: 326, 50: 8451, 51: 1394, 52: 10043, 53: 10172, 54: 6140, 55: 12804, 56: 568, 57: 12035, 58: 3661, 59: 9525, 60: 10049, 61: 8678, 62: 8047, 63: 10358, 64: 13968, 65: 10, 66: 3595, 67: 10049, 68: 3509, 69: 11786, 70: 169, 71: 7973, 72: 1294, 73: 2478, 74: 2793, 75: 11471, 76: 5761, 77: 6647, 78: 14416, 79: 9529, 80: 6567, 81: 14687, 82: 14591, 83: 14795, 84: 14011, 85: 14015, 86: 14889, 87: 14011, 88: 14490, 89: 14013, 90: 9497, 91: 8183, 92: 5145, 93: 13566, 94: 9265, 95: 9265, 96: 14822, 97: 2974

# Evaluation

## NDCG and Cosine Similarity

In [ ]:


train_code_embds = train_code_embds.detach().numpy() if torch.is_tensor(train_code_embds) else train_code_embds
test_doc_embds = test_doc_embds.detach().numpy()if torch.is_tensor(test_doc_embds) else test_doc_embds

ndcga = []
best_emb = []
best_index = []
similarity_scores = []
best_similarity_scores=[]
best_100_indexes = []
best_100_similarities = []

for indx, test_doc_embd in enumerate(test_doc_embds):
# this if checks the size of input query
  if indx in great15:  

    test_doc_embd = np.reshape(test_doc_embd,(1,-1))
    similarities = cosine_similarity(test_doc_embd,train_code_embds)[0]
    similarity_scores.append(similarities)
    best_similarity_scores.append(max(similarities))
    max_sim_index = np.argmax(similarities)
    best_index.append(np.argmax(similarities))
    best_emb.append(train_code_embds[np.argmax(similarities)])
    best_100_indexes.append((np.argsort(similarities)[-50:]))
    best_100_similarities.append([similarities[i] for i in best_100_indexes])
    ndcga.append((ndcg_score((test_doc_embd), ([train_code_embds[max_sim_index]]))))
  
print('Average similarity score',np.sum(best_similarity_scores)/len(best_similarity_scores))
print('Average NDCGA is ',sum(ndcga)/len(ndcga))


Average similarity score 0.9848800048828125
Average NDCGA is  0.9889097737581465


## MRR

In [ ]:
reci_ranks = []
for index, top_100 in enumerate(best_100_indexes):
  if closest_doc_indexes[index] in top_100:
    position_found = list(top_100).index(closest_doc_indexes[index])
    reci_rank = 1 / (position_found+1)
    reci_ranks.append(reci_rank)
  else:
    reci_ranks.append(0)
print('MRR is ',sum(reci_ranks)/len(reci_ranks))

    

length of top 50 is 50
 MRR is --  0.004784113295765302
